# PyCryption Research Tutorial

This notebook serves a technical introduction, demonstration, and collaboration baseline for this repository, PyCryption. It values 'fire-and-forget' systems to allow you to write, and test custom encryption systems.

> Note: Multi-encryption composition systems are currently under research.

---

## 0.0 - Getting Started: Building a Basic Algorithm

In this section we'll exploring how to rapidly test new encryption algorithms without writing tedious harnesses for testing.

### 0.1 Understand the Mental Model

1. **Define your encryption algorithm into a class with two endpoint functions**: `encrypt`, and `decrypt`, that can accept data, and the algorithm context.
2. **Perform the `quick_test`** to ensure successful dataflow.
3. Register other models and algorithms to compare.
4. Refine and/or repeat.

### 0.2 Encryption Helpers and Algorithm Context

An incredibly powerful component, `AlgorithmContext` is a communication interface to your algorithm from the cryptography components you add to your algorithm.

#### 0.21 Key Injection


In [1]:
from lib.notebook import (
    algorithm,
    with_key,
    generate_key,
    AlgorithmContext,
    ComposerSession,
    quick_test,
)

# -----------------------------------------------------------------------------
# Prototype Algorithm Development
# -----------------------------------------------------------------------------
# Use decorators for logistics (key injection, context, metrics).
# Write your own experimental crypto logic in the methods.

KEY = generate_key(32)


@algorithm("XOR-Prototype")
@with_key(KEY)
class XORPrototype:
    """
    Simple XOR-based prototype for testing the framework.
    NOT cryptographically secure - just a development placeholder.
    """

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Prototype: simple repeating-key XOR
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # XOR is symmetric
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))


# Verify the prototype works
quick_test(XORPrototype())

Testing: XOR-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 0.031ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.012ms>
Round-trip successful!


In [2]:
# -----------------------------------------------------------------------------
# Composer Session - Benchmark Prototype vs Production Algorithms
# -----------------------------------------------------------------------------
# Register your prototype alongside proven algorithms from lib/algorithms
# to compare performance and validate behavior.

from lib.notebook import wrap_aes256gcm

# Create session and register algorithms
session = ComposerSession()

# Register our prototype
session.register(XORPrototype())

# Register proven algorithm from lib/algorithms (via adapter)
session.register(wrap_aes256gcm(KEY, name="AES-256-GCM"))

print("Registered algorithms:", session.list_algorithms())

# Test all registered algorithms
print("\n=== Round-trip Tests ===")
test_results = session.test_all()
for name, passed in test_results.items():
    status = "PASS" if passed else "FAIL"
    print(f"  {name}: {status}")

Registered algorithms: ['XOR-Prototype', 'AES-256-GCM']

=== Round-trip Tests ===
  XOR-Prototype: PASS
  AES-256-GCM: PASS


In [3]:
# -----------------------------------------------------------------------------
# Benchmark Prototype Performance
# -----------------------------------------------------------------------------

print("=== Prototype Benchmark (10KB, 50 iterations) ===\n")
comparison = session.compare(data_size=10_000, iterations=50)

print(
    f"{'Algorithm':<20} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput (MB/s)':<15}"
)
print("-" * 65)
for entry in comparison:
    print(
        f"{entry['algorithm']:<20} {entry['avg_encrypt_ms']:<15} {entry['avg_decrypt_ms']:<15} {entry['throughput_mbps']:<15}"
    )

=== Prototype Benchmark (10KB, 50 iterations) ===

Algorithm            Encrypt (ms)    Decrypt (ms)    Throughput (MB/s)
-----------------------------------------------------------------
AES-256-GCM          0.015           0.012           664.89         
XOR-Prototype        1.513           1.518           6.61           


In [4]:
# -----------------------------------------------------------------------------
# Detailed Benchmarks - Scaling Analysis
# -----------------------------------------------------------------------------

print("=== Scaling Analysis ===\n")
benchmarks = session.benchmark_all(
    data_sizes=[100, 1_000, 10_000, 100_000], iterations=20
)

for algo_name, results in benchmarks.items():
    print(f"{algo_name}:")
    print(
        f"  {'Size':<12} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput':<15}"
    )
    print(f"  {'-' * 55}")
    for bench in results["benchmarks"]:
        size_label = f"{bench['size_bytes']:,} B"
        print(
            f"  {size_label:<12} {bench['avg_encrypt_ms']:<15} {bench['avg_decrypt_ms']:<15} {bench['throughput_mbps']:<15} MB/s"
        )

=== Scaling Analysis ===

XOR-Prototype:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.05            0.026           2.02            MB/s
  1,000 B      0.187           0.184           5.34            MB/s
  10,000 B     1.274           1.258           7.85            MB/s
  100,000 B    10.524          10.004          9.5             MB/s
AES-256-GCM:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.01            0.006           10.05           MB/s
  1,000 B      0.007           0.006           136.05          MB/s
  10,000 B     0.01            0.008           1036.27         MB/s
  100,000 B    0.025           0.023           4073.32         MB/s


In [5]:
# -----------------------------------------------------------------------------
# Session Metrics - Aggregated Statistics
# -----------------------------------------------------------------------------

print("=== Session Report ===\n")
report = session.report()

for algo_name, metrics in report.items():
    print(f"{algo_name}:")
    print(
        f"  Operations: {metrics['encrypt_calls']} encrypt, {metrics['decrypt_calls']} decrypt"
    )
    print(
        f"  Avg timing: {metrics['avg_encrypt_ms']:.3f}ms encrypt, {metrics['avg_decrypt_ms']:.3f}ms decrypt"
    )
    print(f"  Total data: {metrics['total_bytes_processed']:,} bytes")
    if metrics["errors"] > 0:
        print(f"  Errors: {metrics['errors']}")
    print()

=== Session Report ===

XOR-Prototype:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 0.032ms encrypt, 0.010ms decrypt
  Total data: 36 bytes

AES-256-GCM:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 1.703ms encrypt, 0.033ms decrypt
  Total data: 52 bytes



---

## 1.0: Example – Adding Cryptographic Capabilities

Beyond a simple byte emitter in the last example, we can easily add new capabilities. Let's take a look at the [Salsa20](https://en.wikipedia.org/wiki/Salsa20) algorithm which was succeeded by its descendent, ChaCha in 2008.


In [8]:
from lib.notebook.decorators import with_kdf, with_salt
import hashlib
from Crypto.Cipher import Salsa20
from lib.notebook import with_metrics


def salsa_kdf(key: bytes, salt: bytes) -> bytes:
    return hashlib.pbkdf2_hmac('sha256', key, salt, 100000, dklen=32)


@algorithm("Salsa20-Prototype")
@with_key(KEY)
@with_kdf("salsa20-kdf", salsa_kdf)
@with_salt("salsa20-salt")
@with_metrics()
class Salsa20Prototype:

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        derived = ctx.derive("salsa20-kdf", "salsa20-salt")
        ctx.layer("salsa20").derived_key = derived  # store for decrypt reference

        cipher = Salsa20.new(derived)

        ctx.layer("salsa20").nonce = cipher.nonce
        return cipher.encrypt(data)

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        derived = ctx.derive("salsa20-kdf", "salsa20-salt")

        nonce = ctx.layer("salsa20").nonce
        cipher = Salsa20.new(derived, nonce)

        return cipher.decrypt(data)


# Verify the prototype works
quick_test(Salsa20Prototype())

Testing: Salsa20-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 35.054ms>
Decrypt: <AlgorithmResult: 18 bytes, 35.419ms>
Round-trip FAILED!
  Expected: b'Hello, PyCryption!'
  Got: b'=\x0e\x9d\xbb\xab\x99\xc0\x99\x19\x00\xdf\x11\xfbs\xd7t\xd3\xef'
